# Summary

This lab will guide you through most of the steps - and pitfalls - of interval velocity estimation, perhaps the most important problem in exploration seismology. You will apply a least squares slowness scan, and then pick RMS slowness using an auto-picker. You will then solve the Dix equation as a least squares problem. Finally, you will test the effects of using regularization and weighting on the Dix inversion.

# Introduction
Exploration seismologists transmit waves through the Earth's surface and record their corresponding traveltimes. In oversimplified terms, the traveltime is the path integral of the interval slowness (1/velocity) over the raypath between source and receiver. "Inversion" of the traveltime for the underlying velocity field is perhaps the most important problem in our branch of geophysics.

Under suitable assumptions, we can directly estimate root-mean-square (RMS) velocity from multioffset seismic data. You have seen the theory in *Basic Earth Imaging* and experimented with it in Lab 2 of this course. However, in order to justify our salaries, we geophysicists must go beyond RMS velocity, and estimate a field of interval velocities, which is needed for seismic depth conversion.


For a stratified ($v(z)$) earth, we can easily write the relationship between RMS velocity ($V$) and interval velocity ($v$):

\begin{equation}\label{eq:vrms}
	V^2(\tau) = \frac{1}{\tau}\ \sum_i v^2_i \Delta\tau_i \qquad (1)
\end{equation}

We say "easily" because the above relationship is implied by the words *root-mean-square*. Equation (1) has an analytic inverse, known as the Dix equation:
\begin{equation}\label{eq:dix}
	v_k^2 = \frac{ \tau_k V_k^2 - \tau_{k-1} V_{k-1}^2 }{ \tau_k - \tau_{k-1} } \qquad (2)
\end{equation}
For a detailed derivation, please see *Basic Earth Imaging*. In practice, the Dix equation often fails to produce a pleasing result, particularly in regions of complex stratigraphy.

In this lab, you will use your expertise in optimization to cast the Dix equation as a least squares problem, in order to more robustly estimate an interval velocity function from a marine CMP gather.


# Your assignment

In this lab, we will work with a CMP gather shown in the Figure below. There are three parts to this lab. First, you will compute a slowness scan using the NMO operator (similar to Lab 2). Second, you will pick the RMS slowness from these scans using a picking program and convert them to RMS velocity. Finally, you will do a Dix inversion of the RMS velocity to convert it to interval velocity. 


In this lab you will use your slowness operator from Lab 2 and we have provided the regularization operators as well as helper functions that will help you to mute (if you find it necessary) and pick the slowness scans. These functions and operators are complete and do not need modification. 

The final part of this lab asks you to perform a weighted, regularized optimization. There are several ways in which you could do this and we have provided you a starter Problem class that can help you with this. If you desire to use this class, you will need to modify it in order to use it for the inversion.

In [2]:
import genericIO
import Grey

# Read in the data
dat = genericIO.defaultIO.getVector("lab4_cmp0.H")
# Get axes
taxis = dat.getHyper().getAxis(1)
xaxis = dat.getHyper().getAxis(2)
nt = taxis.n; ot = taxis.o; dt = taxis.d
nx = xaxis.n; ox = xaxis.o; dx = xaxis.d

grey=Grey.plot(dat,label1="Time (s)",label2="Offset (km)",bclip=-5.0e06,eclip=5.0e06,height=700,width=300,invert_yaxis=True)
grey.image()

:Image   [x,y]   (z)

## Slowness scan optimization

1. Use your NMO operator from Lab2 and estimate the slowness scan by an adjoint only. Plot the scan and comment on its quality and artifacts (if they exist). (Note: you can use the function mute for muting the refractions and direct arrival in the data which can help reduce artifacts. Also, you should compute the envelope of your scan (using the envelope function) before viewing. Both of these functions can be found within the file ``lab4helper.py``).

2. With all the experience you have from previous labs, you know that you can do better than an adjoint. Perform a least-squares optimization to compute the slowness scan using a only a data-fitting term. Plot your results. Do they appear to be better? Why?

3. If you looked at the previous results and thought that regularization is needed, you are right. Redo the previous question but now with a regularization term as well. Feel free to try and experiment with the different regularizations (e.g., identity, gradient, Laplacian, etc.) and values for epsilon. Which regularization seemed to give you the best result? Plot your results and comment on the effects of regularization. 

## Slowness RMS Picking

Now that you managed to produce a nice and clean slowness scan, we want to use it to pick the RMS slownesss. We can use the picker function within ``lab4helper.py`` to do the picking for you. This program picks the maximum of the function within an upper and lower bound. The bounds are computed via

\begin{equation}
  s_{\text{bound}}(\tau) = s_0 + a\tau + b\sqrt{\tau} \pm e
\end{equation}

where $s_0$ defines an intercept of the bound, $a$ and $b$ are parameters that control the slope of the bound and $e$ controls the width of the bounds. The parameters $s_0$, $a$, $b$ and $e$ need to be supplied to the function picker so it can compute the bounds. The picker function will then have four outputs: the slowness function within the bounds, the picked RMS slowness, the times of the picked slowness and finally the amplitudes associated with the picked slownesses.

4. Adjust the bounds parameters until you are happy with the picks. What are the values of the parameters? Convert the picks from slowness to velocity and plot your result.

5. Write a program which applies an NMO correction to the CMP gather using the picked RMS slowness. Run this program using the auto-picked RMS slowness and plot your results. Are all the events flat? How well did the picker do?

## Dix Optimization

6. Before we setup a least-squares optimization, let's try using the Dix equations directly. Write a function that computes the interval velocity using the Dix equation. Run this function on the auto-picked RMS velocity. Plot your results and comment on their accuracy and stability.

7. You decide now that using the Dix equation (i.e., the direct inverse) might not be a good idea for auto-picked RMS velocity. Instead you want to setup the interval velocity estimation as a least-squares optimization from equation (1). What is the model space, data space and operator? (Hint: we want the problem to be linear)

8. Write or use a pre-existing class for the operator you describe in the previous question. Then, run the dot product test on this operator. What are your results?

9. Your experience tells you that we should not use data-fitting optimization only since you already know that the inverse did not work. Therefore, you decide to jump to a regularized optimization right away. Write the necessary code to set up the regularized problem using a regularization operator of your choice. Run the optimization for some iterations and plot the estimated interval velocity (not the model). How well did your optimization do? Why did you get these results?

10. You remember now that your auto-picker also had the amplitudes of the scan as an output and you decide to use these values to weight the data-fitting equation. Using the weight operator class, construct a weight operator with with these values. Then, run another optimization that applies a weighting operator to the data fitting term. You will also want to maintain the regularization term. Run the optimization for some iterations and plot the estimated interval velocity. Did these weights improve the optimization? (**Note**: there are several ways in which you could implement this. One way would be to create a new problem class which we have started for you in the file ``ProblemL2LinearRegWgt.py``.)

## Extra credit

1. Does picking the maximum of the velocity scan really give the RMS velocity? If not, what is the difference between the RMS velocity and what is being measured?

2. Why is using the Dix equation not stable?

3. If you were doing the interval velocity estimation for a whole survey with several CMPs, would constrain the Dix optimization any differently? Why?